# Imports y obtención de archivos

In [ ]:
import pandas as pd
import math

pd.options.display.float_format = '{:20,.2f}'.format

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id1="17SvnFyBNyY018rxrRnOLGp9YaZfeRknp"
downloaded1 = drive.CreateFile({'id': id1})
downloaded1.GetContentFile('googleplaystore.csv')

id2="1DBCzFlJWzdVirjQmoyTaH3GekLTzrznm"
downloaded2 = drive.CreateFile({'id': id2})
downloaded2.GetContentFile('googleplaystore_user_reviews.csv')

# Inicialización de los dataframes

*contiene código importante para algunas de las preguntas*

In [ ]:
ps_apps = pd.read_csv("googleplaystore.csv", skiprows=[10473])

# Saco las apps duplicadas, me quedo con la primera línea (por ninguna razón en particular)
ps_apps = ps_apps.drop_duplicates(subset=["App"], keep="first")

# Casts a categóricas
ps_apps["Type"] = ps_apps["Type"].astype("category")
ps_apps["Content Rating"] = ps_apps["Content Rating"].astype("category")
ps_apps["Android Ver"] = ps_apps["Android Ver"].astype("category")
ps_apps["Installs"] = ps_apps["Installs"].map(lambda x : int(x[:-1].replace(',', '')) if x != "0" else 0)

# La columna de fechas
ps_apps['Last Updated'] = pd.to_datetime(ps_apps['Last Updated'], format="%B %d, %Y")

# lambdas
ps_apps["Genres"] = ps_apps["Genres"].map(lambda x : x.split(";"))
# Esto es sólo para que las categorías tengan nombres más 'amigables'
ps_apps["Category"] = ps_apps["Category"].map(lambda x : x.replace("_", " ").title())
ps_apps["Category"] = ps_apps["Category"].astype("category")

# Conversión de size a float. Hay valores en mb y kb
# Aclaración: esta parte es muy importante para la p28
def size_to_float(sz):
  if sz == "Varies with device":
    return math.nan
  char = sz[-1]
  sz = sz[:-1]
  return float(sz) if char == 'M' else float(sz)/1024
ps_apps["Size"] = ps_apps["Size"].map(size_to_float)

In [ ]:
ps_reviews = pd.read_csv("googleplaystore_user_reviews.csv")
ps_reviews["Sentiment"] = ps_reviews["Sentiment"].astype("category")

# Pregunta #14
[*link al notebook dedicado*](https://colab.research.google.com/drive/1YIEyq9jMWgJF_6bUsBfEDE4TkLKdncX6?usp=drive_link)





> *Top 5 de categorías con mayor cantidad de reviews.*

**Respuesta:**


1. *Game:* 622M
2. *Communication:* 285M
3. *Tools:* 229M
4. *Social:* 227M
5. *Family:* 143M

**Notas:**
- No hay mucho que decir, es bastante lineal el ejercicio

In [ ]:
ps_apps[["Category", "Reviews"]] \
        .groupby(["Category"]) \
        .sum() \
        .nlargest(5, "Reviews")

,Reviews
Category,
Game,622298709
Communication,285811368
Tools,229356578
Social,227927801
Family,143825488


# Pregunta #16
[*link al notebook dedicado*](https://colab.research.google.com/drive/15Of3p9skuvWye0vU1uIFXENbJRwRSTgN?usp=drive_link)

>*Para el rating promedio de cada categoría que no sea 1.9, ¿Cuáles son los 5 promedios más comunes?*

**Respuesta:**

1. avg 4.2 *(15 ocurrencias)*
2. avg 4.1 *(7 ocurrencias)*
3. avg 4.3 *(4 ocurrencias)*
4. avg 4.0 *(4 ocurrencias)* *(empatado en 3°, markdown no me deja poner 2 tercer puestos)*
5. avg 4.4 *(3 ocurrencias)*

**Notas:**

- Los promedios están redondeados a un decimal. Elegí así porque es la misma cantidad de decimales que usa el dataset para los promedios, y para 2 decimales se repetían muy pocos valores
- La única línea que tenía la categoría "1.9" se omitió directamente al cargar el dataset

In [ ]:
ps_apps[["Category", "Rating"]] \
          .groupby(["Category"]) \
          .mean() \
          .Rating \
          .round(1) \
          .value_counts()

4.20    15
4.10     7
4.30     4
4.00     4
4.40     3
Name: Rating, dtype: int64

# Pregunta #28
[*link al notebook dedicado*](https://colab.research.google.com/drive/1cwe6O4J_fAJrI_EYkiWEUyGw-WPge_yd?usp=drive_link)

> *Calcule el tamaño promedio de las aplicaciones por versión de Android, sin tener en cuenta las aplicaciones que varían en tamaño según dispositivo.*

**Respuesta:** *en la salida del bloque abajo de este*

**Notas:**
- La parte *heavy* de este ejercicio (i. e. pasar el *size* a un tipo numérico y filtrar los "*Varies with device*") la hago en el bloque de inicialización del dataframe, me parece mejor práctica tener todo el formato correcto antes de empezar a trabajar
- '*Varies with device*' lo convierto en NaN y el mean me los filtra solo
- Pregunté por slack y me dijeron que, excepto '1.9', el resto de las categorías las podía tomar como buenas así como vienen
- Trabajé todo en Mb proque me parece más acorde a la escala que tienen los tamaños de las apps (prefiero un par de apps con 0.5 Mb a la mayoría con 20000 Kb)

In [ ]:
# "Varies with device" fue mapeado a NaN, el mean los filtra solo

ps_apps[["Size", "Android Ver"]] \
          .groupby(["Android Ver"]) \
          .mean()

,Size
Android Ver,
1.0 and up,3.86
1.5 and up,5.07
1.6 and up,3.12
2.0 and up,6.26
2.0.1 and up,22.22
2.1 and up,5.58
2.2 - 7.1.1,5.10
2.2 and up,8.22
2.3 and up,20.26


# Pregunta #35
[*link al notebook dedicado*](https://colab.research.google.com/drive/1upnXsGtQcHDANmkp-Pg4-gxDAaG3lr2F?usp=drive_link)

> *Correlación entre la polaridad y subjetividad promedio de los comentarios de los juegos cuya última actualización haya sido durante el 2018 y cuyo tamaño sea mayor al tamaño promedio de los juegos de ese año. Ignorar las aplicaciones cuyo tamaño varía con el dispositivo (Varies with device).*

**Respuesta:** 0.46

**Notas:**
- Se ignoran los valores de `ps_reviews` con polaridad exactamente igual a 0 porque descubrí con las [visus](https://colab.research.google.com/drive/1Izta0ha_L23-M1VctJ-p5mVxNKUUFYlN) que es altamente probable que la mayoría de los 0.0 hayan sido asignados por error (tldr; la distribución es muy parecida a una normal pero con un pico enorme en 0, y si ignoro los 0 la distribución es más sensible)
- Como mencioné en la pregunta anterior, el formateo de las fechas y el tamaño se hace durante la etapa de inicialización del df
- Sorry por la cantidad absurda de comentarios en el código pero es que si no me pierdo (?

In [ ]:
#inicializar los df con los datos que necesito
# De las reviews, las columnas de polaridad y subjetividad, agrupadas por app, saco promedio
# Ignoro los valores 0 de polaridad porque están mal, ver las visus
df_right = (ps_reviews[ps_reviews.Sentiment_Polarity != 0]
          [["App", "Sentiment_Polarity", "Sentiment_Subjectivity"]]
          .groupby(["App"])
          .mean())

# De las apps, los que tengan año de last update en 2018 y categoría game,
# Las columnas 'app' y 'size'
df_left = (ps_apps[(ps_apps["Last Updated"].dt.year == 2018) & (ps_apps["Category"] == "Game")]
            [["App", "Size"]])

# Hago el merge
ps_apps.reset_index()
df_right.reset_index()
df_q35 = pd.merge(
                df_left,
                df_right,
                on="App",
                how="inner")

# Saco tamaño promedio, correlación de polaridad y subjetividad de los que superen el promedio
mean_sz = df_q35["Size"].mean()
df_q35[df_q35["Size"] > mean_sz] \
      [["Sentiment_Polarity", "Sentiment_Subjectivity"]] \
      .corr()

,Sentiment_Polarity,Sentiment_Subjectivity
Sentiment_Polarity,1.00,0.46
Sentiment_Subjectivity,0.46,1.00


# Pregunta #37
[*link al notebook dedicado*](https://colab.research.google.com/drive/1tm869nXurrKNNajHnFkqerh3LYoAYT9Y?usp=drive_link)

> *Devolver las categorías que utilicen más caracteres distintos para sus apps, y las que menos caracteres distintos usan. ¿Cuáles son los caracteres que se comparten entre ambas?*

**Respuesta:**
- *Con más*: Family (211 caracteres)
- *Con menos*: Beauty (39 caracteres)
- *Compartidos*: ver salida (Son todos los de *Beauty*)

**Notas:**
- Me pareció más sencillo definir la función de contar caracteres aquí que meter las librerías de nlp *solamente* para este pedacito de esta pregunta (además de que me convenía empezar de una a trabajar con sets para la intersección).
- Decidi contar mayúsculas y minúsculas como caracteres iguales
- Me fijé de que ambos valores fueran únicos (además el mayor le saca bastante al segundo)

In [ ]:
# Esta función me parece un poco bruta, pero sirve
def charset(series):
  cset = set({})
  for item in series:
    for letter in item:
      cset.add(letter.lower())
  return cset

df37 = (ps_apps.groupby("Category")
            .agg({"App": charset})
            .reset_index()
            .rename(columns={"App": "Characters"}))
df37["Character_Count"] = df37.apply(lambda row: len(row["Characters"]), axis=1)

top = df37.nlargest(1, "Character_Count")
bottom = df37.nsmallest(1, "Character_Count")
shared = top.iloc[0]["Characters"].intersection(bottom.iloc[0]["Characters"])

# A partir de aquí es para que la salida me quede más bonita
topcat = top.iloc[0]["Category"]
bottomcat = bottom.iloc[0]["Category"]

topcount = top.iloc[0]["Character_Count"]
bottomcount = bottom.iloc[0]["Character_Count"]

print(f"==> Categoría con más caracteres únicos: {topcat} ({topcount} caracteres)")
print(f"==> Categoría con menos caracteres únicos: {bottomcat} ({bottomcount} caracteres)")
print(f"==> {len(shared)} caracteres compartidos: {shared}")